# Disinformation

An example where an individual spreads a fake news among others and they discuss whether it is true or not, as well as some methods of preventing the spread of these.

## Setup

### Import libraries

In [ ]:
import os
import random
import datetime
import numpy as np
import concurrent.futures
import sentence_transformers
import matplotlib.pyplot as plt
import collections

from typing import Callable
from IPython import display

from concordia.utils import plotting
from concordia.clocks import game_clock
from concordia.agents import basic_agent
from concordia.utils import html as html_lib
from concordia.environment import game_master
from concordia.language_model import gpt_model
from concordia.document import interactive_document
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import blank_memories
from concordia.components import agent as agent_components
from concordia.components import agent as components
from concordia.factory.environment import basic_game_master
from concordia.associative_memory import associative_memory
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.utils import measurements as measurements_lib
from concordia.components import game_master as gm_components
from concordia.thought_chains import thought_chains as thought_chains_lib
from concordia.environment.scenes import conversation as conversation_scene
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others

### Sentence Encoder Setup


In [ ]:
st_model = sentence_transformers.SentenceTransformer(
    'sentence-transformers/all-mpnet-base-v2')
embedder = lambda x: st_model.encode(x, show_progress_bar=False)

### Language Model Initialization

In [3]:
GPT_API_KEY = 'lm-studio' #@param {type: 'string'}
GPT_MODEL_NAME = 'LM Studio Community/Meta-Llama-3-8B-Instruct-GGUF' #@param {type: 'string'}

if not GPT_API_KEY:
  raise ValueError('GPT_API_KEY is required.')

model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY,
                                   model_name=GPT_MODEL_NAME)

## Configuring the generic knowledge of players and GM

In [ ]:
# Define all the characters of the environment
CHARACTERS = ['Alice', 'David', 'Emma', 'Michael']
WHO_SPREADS_NEWS = np.random.choice(CHARACTERS)

# Define the fake news
FAKE_NEWS = "First Manned Mission to Mars Scheduled for Next Month!"

shared_memory = [
    f' You are having a casual conversation with some friends when {WHO_SPREADS_NEWS} suddenly'
    f' shares a surprising piece of news: "{FAKE_NEWS}". Then all of you discuss whether the news'
    ' is true or false, evaluating the information based on your knowledge, the'
    ' credibility of the source, and any verification tools you have. This setting'
    ' provides a realistic scenario for observing the spread of fake news and testing'
    ' strategies to prevent it.'
]

# Context that all characters and GM share
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memory)
    + '\n'
    + 'Summary:'
)
print(shared_context)

importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

### Make the clock

In [5]:
UPDATE_INTERVAL = datetime.timedelta(seconds=10)

SETUP_TIME = datetime.datetime(hour=8, year=2024, month=6, day=1)

START_TIME = datetime.datetime(hour=14, year=2024, month=7, day=1)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[UPDATE_INTERVAL, datetime.timedelta(seconds=10)])

NUM_ROUNDS = 3

## Functions to build the agents

It might be convenient to highlight the difference between a **blank memory** and a **formative memory**:

* Blank memory: refers to a state where no prior information or data is stored (essentially an empty memory state).
* Formative memory: consists of past experiences and information that shape the current state of the agent (it includes all the data that the agent has acquired and learned from over time).

In [6]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [7]:
def build_agent(agent_config,
                player_names: list[str],
                measurements: measurements_lib.Measurements | None = None):

  mem = formative_memory_factory.make_memories(agent_config)

  agent_name = agent_config.name
  instructions = generic_components.constant.ConstantComponent(
      state=(
          f'The instructions for how to play the role of {agent_name} are as '
          'follows. This is a social science experiment studying how well you '
          f'play the role of a character named {agent_name}. The experiment '
          'is structured as a tabletop roleplaying game (like dungeons and '
          'dragons). However, in this case it is a serious social science '
          'experiment and simulation. The goal is to be realistic. It is '
          f'important to play the role of a person like {agent_name} as '
          f'accurately as possible, i.e., by responding in ways that you think '
          f'it is likely a person like {agent_name} would respond, and taking '
          f'into account all information about {agent_name} that you have. '
          'Always use third-person limited perspective.'
      ),
      name='role playing instructions\n')

  if agent_config.extras.get('spreads news', False):
    fact = generic_components.constant.ConstantComponent(
        state=f'{agent_name} spreads the fake news: "{FAKE_NEWS}"', name='fact')
  else:
    fact = generic_components.constant.ConstantComponent(
        state=f'{agent_name} tries to check the veracity of the news.', name='fact')

  time = generic_components.report_function.ReportFunction(
      name='Current time',
      function=clock.current_time_interval_str,
  )

  current_obs = components.observation.Observation(
            agent_name=agent_config.name,
      clock_now=clock.now,
      memory=mem,
      timeframe=clock.get_step_size(),
      component_name='current observations',
  )
  somatic_state = components.somatic_state.SomaticState(
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
  )
  summary_obs = components.observation.ObservationSummary(
      agent_name=agent_config.name,
      model=model,
      clock_now=clock.now,
      memory=mem,
      components=[current_obs],
      timeframe_delta_from=datetime.timedelta(hours=4),
      timeframe_delta_until=datetime.timedelta(hours=1),
      component_name='summary of observations',
  )

  self_perception = components.self_perception.SelfPerception(
      name=f'answer to what kind of person is {agent_config.name}',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
  )
  relevant_memories = components.all_similar_memories.AllSimilarMemories(
      name='relevant memories',
      model=model,
      memory=mem,
      agent_name=agent_name,
      components=[summary_obs, self_perception],
      clock_now=clock.now,
      num_memories_to_retrieve=25,
      verbose=False,
  )
  situation_perception = components.situation_perception.SituationPerception(
      name=(f'answer to what kind of situation is {agent_config.name} in ' +
            'right now'),
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      components=[current_obs, somatic_state, summary_obs],
      clock_now=clock.now,
  )
  person_by_situation = components.person_by_situation.PersonBySituation(
      name=(f'answer to what would a person like {agent_config.name} do in a ' +
            'situation like this'),
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      components=[self_perception, situation_perception],
      verbose=True,
  )

  persona = generic_components.sequential.Sequential(
      name='persona',
      components=[
          self_perception,
          situation_perception,
          person_by_situation,
      ]
  )

  justification_components = components.justify_recent_voluntary_actions
  justification = justification_components.JustifyRecentVoluntaryActions(
      name='justification',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      components=[persona, somatic_state, relevant_memories],
      clock_now=clock.now,
      verbose=True,
  )
  reflection = components.dialectical_reflection.DialecticalReflection(
      name='reflection',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      intuition_components=[self_perception, justification],
      thinking_components=[persona],
      clock_now=clock.now,
      num_memories_to_retrieve=5,
      verbose=True,
  )

  initial_goal_component = generic_components.constant.ConstantComponent(
      state=agent_config.goal, name='overarching goal')
  plan = components.plan.SimPlan(
      model,
      mem,
      agent_config.name,
      clock_now=clock.now,
      components=[instructions,
                  fact,
                  initial_goal_component,
                  relevant_memories,
                  persona,
                  justification],
      goal=person_by_situation,
      horizon='the next hour',
      verbose=True,
  )

  goal_metric = goal_achievement.GoalAchievementMetric(
      model=model,
      player_name=agent_config.name,
      player_goal=agent_config.goal,
      clock=clock,
      name='Goal Achievement',
      measurements=measurements,
      channel='goal_achievement',
      verbose=False,
  )
  morality_metric = common_sense_morality.CommonSenseMoralityMetric(
      model=model,
      player_name=agent_config.name,
      clock=clock,
      name='Morality',
      verbose=False,
      measurements=measurements,
      channel='common_sense_morality',
  )
  agent = basic_agent.BasicAgent(
      model,
      agent_name=agent_config.name,
      clock=clock,
      verbose=True,
      components=[instructions,
                  fact,
                  persona,
                  justification,
                  reflection,
                  plan,
                  time,
                  relevant_memories,
                  current_obs,
                  goal_metric,
                  morality_metric],
      update_interval = UPDATE_INTERVAL
  )
  reputation_metric = opinion_of_others.OpinionOfOthersMetric(
      model=model,
      player_name=agent_config.name,
      player_names=player_names,
      context_fn=agent.state,
      clock=clock,
      name='Opinion',
      verbose=False,
      measurements=measurements,
      channel='opinion_of_others',
      question='What is {opining_player}\'s opinion of {of_player}?',
  )
  agent.add_component(reputation_metric)
  return agent, mem

## Configure and build the agents

In [8]:
# Define a function to generate random traits for an agent
TRAIT_LEVELS = ["low", "medium", "high"] # range of possible levels for traits

def make_random_traits() -> str:
  return str({
      "extraversion": np.random.choice(TRAIT_LEVELS),
      "neuroticism": np.random.choice(TRAIT_LEVELS),
      "openness": np.random.choice(TRAIT_LEVELS),
      "conscientiousness": np.random.choice(TRAIT_LEVELS),
      "agreeableness": np.random.choice(TRAIT_LEVELS),
      "susceptibility": np.random.choice(TRAIT_LEVELS),
      "critical thinking": np.random.choice(TRAIT_LEVELS),
  })


# Create player configurations, each represented by an AgentConfig object
player_configs = [
    formative_memories.AgentConfig(
        name=CHARACTERS[0],
        gender='female',
        goal=f"Figure out if the news spread by {WHO_SPREADS_NEWS} are true or not",
        #context=(motives[WHO_FOUND_BODY] + ' ' +
        #        motives_to_investigate[WHO_FOUND_BODY] + ' ' +
        #         knowledge_of_scandal.get(WHO_FOUND_BODY, '')),
        traits = make_random_traits(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
        extras={'spreads news': CHARACTERS[0] == WHO_SPREADS_NEWS,
                'political_ideology': 'liberal'},
    ),
    formative_memories.AgentConfig(
        name=CHARACTERS[1],
        gender='male',
        goal=f"Figure out if the news spread by {WHO_SPREADS_NEWS} are true or not",
        #context=(motives[WHO_FOUND_BODY] + ' ' +
        #        motives_to_investigate[WHO_FOUND_BODY] + ' ' +
        #         knowledge_of_scandal.get(WHO_FOUND_BODY, '')),
        traits = make_random_traits(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
        extras={'spreads news': CHARACTERS[1] == WHO_SPREADS_NEWS,
                'political_ideology': 'conservative'},
    ),
    formative_memories.AgentConfig(
        name=CHARACTERS[2],
        gender='female',
        goal=f"Figure out if the news spread by {WHO_SPREADS_NEWS} are true or not",
        #context=(motives[WHO_FOUND_BODY] + ' ' +
        #        motives_to_investigate[WHO_FOUND_BODY] + ' ' +
        #         knowledge_of_scandal.get(WHO_FOUND_BODY, '')),
        traits = make_random_traits(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
        extras={'spreads news': CHARACTERS[2] == WHO_SPREADS_NEWS,
                'political_ideology': 'moderate'},
    ),
    formative_memories.AgentConfig(
        name=CHARACTERS[3],
        gender='male',
        goal=f"Figure out if the news spread by {WHO_SPREADS_NEWS} are true or not",
        #context=(motives[WHO_FOUND_BODY] + ' ' +
        #        motives_to_investigate[WHO_FOUND_BODY] + ' ' +
        #         knowledge_of_scandal.get(WHO_FOUND_BODY, '')),
        traits = make_random_traits(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
        extras={'spreads news': CHARACTERS[3] == WHO_SPREADS_NEWS,
                'political_ideology': 'libertarian'},
    ),
]

In [ ]:
NUM_PLAYERS = len(player_configs)
player_names = [player.name for player in player_configs][:NUM_PLAYERS]
measurements = measurements_lib.Measurements()

players = []
memories = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent, mem in pool.map(build_agent,
                             player_configs[:NUM_PLAYERS],
                             # All players get the same `player_names`.
                             [player_names] * NUM_PLAYERS,
                             # All players get the same `measurements` object.
                             [measurements] * NUM_PLAYERS):
    players.append(agent)
    memories[agent._agent_name] = mem

### Summarise the perspective of each player

In [ ]:
player_logs = []
player_log_names = []
for player in players:
  name = player.name
  detailed_story = '\n'.join(memories[player.name].retrieve_recent(
      k=1000, add_time=True))
  summary = player.state().splitlines()

  all_player_mem = memories[player.name].retrieve_recent(k=1000, add_time=True)
  all_player_mem = ['Player state:', summary, 'Memories:'] + all_player_mem
  player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
  player_logs.append(player_html)
  player_log_names.append(f'{name}')

tabbed_html = html_lib.combine_html_pages(
    player_logs,
    player_log_names,
    summary='',
    title='Backstory of the players',
)

tabbed_html = html_lib.finalise_html(tabbed_html)
display.HTML(tabbed_html)

## Build GM

In [ ]:
game_master_memory = associative_memory.AssociativeMemory(
   sentence_embedder=embedder,
   importance=importance_model_gm.importance,
   clock=clock.now)

In [ ]:
# Create components of the Game Master
player_names = [player.name for player in players]

extra_premise = generic_components.constant.ConstantComponent(
    state=('As a result of the snow storm, the Midnight Express has had to ' +
           'stop in the middle of nowhere. The train cannot move till ' +
           'morning. The snow storm makes it impossible to leave the train.' +
           'It is also impossible for outsiders like the police to reach the ' +
           'train.'),
    name='Premise')
scenario_knowledge = generic_components.constant.ConstantComponent(
    state=' '.join(shared_memory),
    name='Background')
informant = generic_components.constant.ConstantComponent(
    state=f'{WHO_SPREADS_NEWS} has shared some exciting news with the other participants',
    name='Fact')

time_display=generic_components.report_function.ReportFunction(
    name='Current time',
    function=clock.current_time_interval_str)

player_status = gm_components.player_status.PlayerStatus(
    clock_now=clock.now,
    model=model,
    memory=game_master_memory,
    player_names=player_names)

convo_externality = gm_components.conversation.Conversation(
    players=players,
    model=model,
    memory=game_master_memory,
    clock=clock,
    burner_memory_factory=blank_memory_factory,
    components=[informant, player_status],
    allow_self_talk=True,
    cap_nonplayer_characters=2,
    shared_context=shared_context,
    verbose=True,
)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players=players,
    model=model,
    memory=game_master_memory,
    clock_now=clock.now,
    verbose=False,
    components=[player_status]
)

In [ ]:
# Create the game master's thought chain
account_for_agency_of_others = thought_chains_lib.AccountForAgencyOfOthers(
    model=model, players=players, verbose=False)
thought_chain = [
    thought_chains_lib.extract_direct_quote,
    thought_chains_lib.attempt_to_most_likely_outcome,
    thought_chains_lib.result_to_effect_caused_by_active_player,
    account_for_agency_of_others,
    thought_chains_lib.restore_direct_quote,
]


In [ ]:
# Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    update_thought_chain=thought_chain,
    components=[
        extra_premise,
        scenario_knowledge,
        informant,
        player_status,
        convo_externality,
        direct_effect_externality,
        time_display,
    ],
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
)

### The RUN

In [ ]:
clock.set(START_TIME)

In [ ]:
scenario_premise = (
    'Alice, David, Emma, and Michael are in the Bravo Bar in a Friday night. ' +
    f'They are having a casual conversation when {WHO_SPREADS_NEWS} suddenly shares ' +
    f'a pieces of news: "{FAKE_NEWS}".  They fall silent as everyone processes this ' +
    'unexpected announcement. Reactions range from  disbelief to curiosity, ' +
    'and it quickly becomes clear that doubts are surfacing among the group. ' +
    'As the initial shock wears off, the group begins to discuss whether the news is true or false. ' +
    'They evaluate the information based on their own knowledge, the credibility of the source, and ' +
    'any verification tools they have at their disposal.'
)

fake_news_info = f'{WHO_SPREADS_NEWS} shared the following piece of news: "{FAKE_NEWS}".'
discussion_info = (
    'The group quickly becomes divided, with some members believing the news and others remaining skeptical. '
    'Everyone starts discussing the credibility of the information, debating its truthfulness based on their knowledge, '
    'the reliability of {WHO_SPREADS_NEWS}, and any verification methods they might use.'
)

game_master_memory.add(scenario_premise)
for player in players:
    # Provide general information about each player
    player_info = f'{player.name} is in the Bravo Bar.'
    player.observe(player_info)
    game_master_memory.add(player_info)

    # Provide the scenario premise to all players
    player.observe(scenario_premise)
    game_master_memory.add(scenario_premise)

    # Provide additional information based on player roles
    if player.name == WHO_SPREADS_NEWS:
        player.observe(fake_news_info)
        game_master_memory.add(fake_news_info)

    # All players will observe the discussion information
    player.observe(discussion_info)
    game_master_memory.add(discussion_info)

In [ ]:
episode_length = 4  # @param {type: 'integer'}
for _ in range(episode_length):
  env.step()

## Summary and analysis of the episode

In [ ]:
# Metrics plotting
group_by = collections.defaultdict(lambda: 'player')
group_by['opinion_of_others'] = 'of_player'

available_channels = list(measurements.available_channels())

fig, ax = plt.subplots(1, len(available_channels), figsize=(6, 2))
tb = [channel for channel in available_channels]
for idx, channel in enumerate(available_channels):
  plotting.plot_line_measurement_channel(
      measurements,
      channel,
      group_by=group_by[channel],
      xaxis='time_str',
      ax=ax[idx])
  ax[idx].set_title(channel)

fig.set_constrained_layout(constrained=True)

In [ ]:
# Summarize the entire story.
all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

detailed_story = '\n'.join(all_gm_memories)
print('len(detailed_story): ', len(detailed_story))
# print(detailed_story)

episode_summary = model.sample_text(
    f'Sequence of events:\n{detailed_story}'+
    '\nNarratively summarize the above temporally ordered ' +
    'sequence of events. Write it as a news report. Summary:\n',
     max_tokens=8000, terminators=())
print(episode_summary)

In [ ]:
# Summarise the perspective of each player
player_logs = []
player_log_names = []
for player in players:
  name = player.name
  detailed_story = '\n'.join(memories[player.name].retrieve_recent(
      k=1000, add_time=True))
  summary = ''
  summary = model.sample_text(
      f'Sequence of events that happened to {name}:\n{detailed_story}'
      '\nWrite a short story that summarises these events.\n'
      ,
       max_tokens=8000, terminators=())

  all_player_mem = memories[player.name].retrieve_recent(k=1000, add_time=True)
  all_player_mem = ['Summary:', summary, 'Memories:'] + all_player_mem
  player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
  player_logs.append(player_html)
  player_log_names.append(f'{name}')

In [ ]:
history_sources = [
    env,
]
histories_html = [
    html_lib.PythonObjectToHTMLConverter(history.get_history()).convert()
    for history in history_sources]
histories_names = [history.name for history in history_sources]

In [ ]:
gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

tabbed_html = html_lib.combine_html_pages(
    histories_html + [gm_mem_html] + player_logs,
    histories_names + ['GM'] + player_log_names,
    summary=episode_summary,
    title='Murder mystery on a train',
)

tabbed_html = html_lib.finalise_html(tabbed_html)

In [ ]:
# Display the HTML visualization of the log
display.HTML(tabbed_html)